In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
from transformers.modeling_utils import PreTrainedModel

In [5]:
# Define configurations for loading the model
max_seq_length = 2048
dtype = None  # Automatically choose the best data type (float16, bfloat16, etc.)
load_in_4bit = True  # Enable 4-bit quantization to reduce memory usage

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

==((====))==  Unsloth 2025.7.3: Fast Llama patching. Transformers: 4.53.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [21]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
   tokenizer,
   chat_template = "llama-3.2",
)
# Set the PAD token to be the same as the EOS token to avoid tokenization issues
tokenizer.pad_token = tokenizer.eos_token
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
   {"role": "user", "content": "What is the primary function of the product Damini of AtoZ company?"}]
# Tokenize the user input with the chat template
inputs = tokenizer.apply_chat_template(
   messages,
   tokenize=True,
   add_generation_prompt=True,
   return_tensors="pt",
   padding=True,  # Add padding to match sequence lengths
).to("cuda")

attention_mask = inputs != tokenizer.pad_token_id

outputs = model.generate(
   input_ids=inputs,
   attention_mask=attention_mask,
   max_new_tokens=64,
   use_cache=True,  # Use cache for faster token generation
   temperature=0.6,  # Controls randomness in responses
   min_p=0.1,  # Set minimum probability threshold for token selection
)

# Decode the generated tokens into human-readable text
text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(text)

system

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

user

What is the primary function of the product Damini of AtoZ company?assistant

I am unable to verify the primary function of the product Damini of AtoZ company.


In [ ]:
#from huggingface_hub import login

In [ ]:
#login()

In [ ]:
#dataset.push_to_hub("dharsandip/sandip_faq_data")

In [22]:
# Prepare the Training Dataset

from datasets import load_dataset  # Load datasets from Hugging Face Hub

# Load a dataset
#dataset = load_dataset(dataset, split="train")
#dataset = load_dataset("dharsandip/sandip_faq_data", split="train")

In [23]:
json_data = [
    {
        "conversation": [
            {"role": "user", "content": "What is the primary function of the product Damini of AtoZ company?"},
            {"role": "assistant", "content": "This product is a cloud-based software solution that allows users to manage and analyze large datasets through advanced machine learning algorithms."}
        ]
    },
    {
        "conversation": [
            {"role": "user", "content": "Who is the target audience for this product Damini"},
            {"role": "assistant", "content": "Data scientists, analysts, researchers and businesses looking to extract valuable insights from complex datasets."}
        ]
    },
    {
        "conversation": [
            {"role": "user", "content": "What are the key features of the product Damini?"},
            {"role": "assistant", "content": "Data ingestion from diverse sources, data cleaning and pre-processing tools, advanced statistical analysis, predictive modeling, visualization capabilities, and customizable dashboards."}
        ]
    },
    {
        "conversation": [
            {"role": "user", "content": "What type of data formats does the product Damini support?"},
            {"role": "assistant", "content": "CSV, Excel, JSON, XML, SQL databases, and various API integrations."}
        ]
    },
    {
        "conversation": [
            {"role": "user", "content": "What are the different deployment options available for Damini?"},
            {"role": "assistant", "content": "Cloud-based subscription model with access through a web interface, on-premises deployment for high security needs."}
        ]
    },
    {
        "conversation": [
            {"role": "user", "content": "What are the system requirements for using the product Damini?"},
            {"role": "assistant", "content": "Minimum system specifications including required operating systems, memory, and network bandwidth are specified on the product site. Please check them."}
        ]
    },

     {
        "conversation": [
            {"role": "user", "content": "How is the product Damini priced?"},
            {"role": "assistant", "content": "Based on a tiered subscription model with varying features and data storage capacity."}
        ]
    },
     {
        "conversation": [
            {"role": "user", "content": "What are the payment options for Damini?"},
            {"role": "assistant", "content": "Credit card, bank transfer, purchase orders for enterprise clients."}
        ]
    },
     {
        "conversation": [
            {"role": "user", "content": "Is there a free trial available for this product Damini?"},
            {"role": "assistant", "content": "Yes, a limited-time free trial with access to core functionalities is available for new users."}
        ]
    },
     {
        "conversation": [
            {"role": "user", "content": "How can I access customer support for product Damini?"},
            {"role": "assistant", "content": "Through our online customer support portal, email, or phone during the business hours."}
        ]
    },
     {
        "conversation": [
            {"role": "user", "content": "What types of support are offered for Damini?"},
            {"role": "assistant", "content": "User Documentations, tutorials, technical troubleshooting and personalized assistance from our dedicated support team."}
        ]
    },
     {
        "conversation": [
            {"role": "user", "content": "How does the product Damini protect my data?"},
            {"role": "assistant", "content": "Industry-standard encryption protocols, strict data privacy policies, and compliance with relevant data protection regulations."}
        ]
    },
     {
        "conversation": [
            {"role": "user", "content": "What are the data residency options for Damini?"},
            {"role": "assistant", "content": "Ability to store data in specific geographic locations based on customer requirements."}
        ]
    },
     {
        "conversation": [
            {"role": "user", "content": "What are the limitations of the product Damini?"},
            {"role": "assistant", "content": "We have mentioned any specific data size restrictions, complex modeling limitations, or features not yet available etc. on the product site. Please check there."}
        ]
    },
     {
        "conversation": [
            {"role": "user", "content": "How can I provide feedback on the product Damini?"},
            {"role": "assistant", "content": "Through the feedback section on the user interface, dedicated feedback channels, or direct contact with the support team."}
        ]
    },
]
json_data

[{'conversation': [{'role': 'user',
    'content': 'What is the primary function of the product Damini of AtoZ company?'},
   {'role': 'assistant',
    'content': 'This product is a cloud-based software solution that allows users to manage and analyze large datasets through advanced machine learning algorithms.'}]},
 {'conversation': [{'role': 'user',
    'content': 'Who is the target audience for this product Damini'},
   {'role': 'assistant',
    'content': 'Data scientists, analysts, researchers and businesses looking to extract valuable insights from complex datasets.'}]},
 {'conversation': [{'role': 'user',
    'content': 'What are the key features of the product Damini?'},
   {'role': 'assistant',
    'content': 'Data ingestion from diverse sources, data cleaning and pre-processing tools, advanced statistical analysis, predictive modeling, visualization capabilities, and customizable dashboards.'}]},
 {'conversation': [{'role': 'user',
    'content': 'What type of data formats do

In [24]:
from datasets import Dataset
dataset = Dataset.from_list(json_data)
dataset

Dataset({
    features: ['conversation'],
    num_rows: 15
})

In [17]:
type(dataset)

datasets.arrow_dataset.Dataset

In [25]:
# Format Prompts
# Once the dataset is prepared, we need to ensure that the data is structured correctly to be used by the model. For this, we apply
# the appropriate chat template (we have used the Llama-3.2 format.) using the get_chat_template function. This function basically prepares
# the tokenizer with the Llama-3.2 chat format for conversation-style fine-tuning

from unsloth.chat_templates import get_chat_template

# Apply the Llama-3.2 chat template to the tokenizer
tokenizer = get_chat_template(
    tokenizer,  # Tokenizer being used
    chat_template="llama-3.2",  # The chat template format
)

# Function to format the conversation data into tokenized text
def formatting_prompts_func(examples):
    convos = examples["conversation"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)

Model does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [26]:
dataset['text']

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is the primary function of the product Damini of AtoZ company?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThis product is a cloud-based software solution that allows users to manage and analyze large datasets through advanced machine learning algorithms.<|eot_id|>',
 '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWho is the target audience for this product Damini<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nData scientists, analysts, researchers and businesses looking to extract valuable insights from complex datasets.<|eot_id|>',
 '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nTo

In [28]:
# Print an item in its original conversation format
print(dataset[0]["conversation"])

# Print the same item in its formatted text format
print(dataset[0]["text"])

[{'content': 'What is the primary function of the product Damini of AtoZ company?', 'role': 'user'}, {'content': 'This product is a cloud-based software solution that allows users to manage and analyze large datasets through advanced machine learning algorithms.', 'role': 'assistant'}]
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the primary function of the product Damini of AtoZ company?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

This product is a cloud-based software solution that allows users to manage and analyze large datasets through advanced machine learning algorithms.<|eot_id|>


In [27]:
# Setup and Configure the Trainer
# configure the fine-tuning process using Hugging Face’s SFTTrainer. It automates key tasks like tokenization, batching, and optimization,
# making fine-tuning easier. SFTTrainer works efficiently with Unsloth, reducing VRAM usage and speeding up training

from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

# Define training configurations
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=2,
    packing=False,

    args=TrainingArguments(
        per_device_train_batch_size=2,  # Number of examples per GPU batch
        gradient_accumulation_steps=4,  # Accumulate gradients over 4 batches before updating model
        warmup_steps=5,  # Number of warmup steps for learning rate schedule
        num_train_epochs=15,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,  # Log training metrics after every step
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",  # Linear decay of learning rate
        seed=3407,
        output_dir="outputs",  # Directory to save model checkpoints
        report_to="none",  # Use this for WandB etc

    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/15 [00:00<?, ? examples/s]

ValueError: You cannot perform fine-tuning on purely quantized models. Please attach trainable adapters on top of the quantized model to correctly perform fine-tuning. Please see: https://huggingface.co/docs/transformers/peft for more details

In [31]:
# Define configurations for loading the model
max_seq_length = 2048
dtype = None  # Automatically choose the best data type (float16, bfloat16, etc.)
load_in_4bit = True  # Enable 4-bit quantization to reduce memory usage

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    lora_alpha=16,
    lora_dropout=0.1,
    lora_r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "linear",
    ],
)

# // Ensure the model with LoRA adapters is used in the SFTTrainer
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

# Define training configurations
trainer = SFTTrainer(
    model=model,  # Ensure the model with LoRA adapters is used here
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=2,
    packing=False,

    args=TrainingArguments(
        per_device_train_batch_size=2,  # Number of examples per GPU batch
        gradient_accumulation_steps=4,  # Accumulate gradients over 4 batches before updating model
        warmup_steps=5,  # Number of warmup steps for learning rate schedule
        num_train_epochs=15,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,  # Log training metrics after every step
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",  # Linear decay of learning rate
        seed=3407,
        output_dir="outputs",  # Directory to save model checkpoints
        report_to="none",  # Use this for WandB etc

    ),
)

==((====))==  Unsloth 2025.7.3: Fast Llama patching. Transformers: 4.53.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: You added custom modules, but Unsloth hasn't optimized for this.
Beware - your finetuning might be noticeably slower!


TypeError: LoraConfig.__init__() got an unexpected keyword argument 'lora_r'

In [22]:
import unsloth

In [24]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 539.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [3]:
from unsloth import FastLanguageModel
import torch

In [35]:
# Define configurations for loading the model
max_seq_length = 2048
dtype = None  # Automatically choose the best data type (float16, bfloat16, etc.)
load_in_4bit = True  # Enable 4-bit quantization to reduce memory usage

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    lora_alpha=16,
    lora_dropout=0.1,
    r=16, # Corrected keyword argument
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "linear",
    ],
)

==((====))==  Unsloth 2025.7.3: Fast Llama patching. Transformers: 4.53.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: You added custom modules, but Unsloth hasn't optimized for this.
Beware - your finetuning might be noticeably slower!


Unsloth 2025.7.3 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [6]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
   tokenizer,
   chat_template = "llama-3.2",
)
# Set the PAD token to be the same as the EOS token to avoid tokenization issues
tokenizer.pad_token = tokenizer.eos_token
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
   {"role": "user", "content": "What is the primary function of the product Damini of AtoZ company?"}]
# Tokenize the user input with the chat template
inputs = tokenizer.apply_chat_template(
   messages,
   tokenize=True,
   add_generation_prompt=True,
   return_tensors="pt",
   padding=True,  # Add padding to match sequence lengths
).to("cuda")

attention_mask = inputs != tokenizer.pad_token_id

outputs = model.generate(
   input_ids=inputs,
   attention_mask=attention_mask,
   max_new_tokens=64,
   use_cache=True,  # Use cache for faster token generation
   temperature=0.6,  # Controls randomness in responses
   min_p=0.1,  # Set minimum probability threshold for token selection
)

# Decode the generated tokens into human-readable text
text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(text)

system

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

user

What is the primary function of the product Damini of AtoZ company?assistant

I'm unable to verify the primary function of the product 'Damini' of AtoZ company.


In [8]:
# Prepare the Training Dataset

from datasets import load_dataset  # Load datasets from Hugging Face Hub

# Load a dataset
#dataset = load_dataset(dataset, split="train")
#dataset = load_dataset("dharsandip/sandip_faq_data", split="train")

In [10]:
json_data = [
    {
        "conversation": [
            {"role": "user", "content": "What is the primary function of the product Damini of AtoZ company?"},
            {"role": "assistant", "content": "This product is a cloud-based software solution that allows users to manage and analyze large datasets through advanced machine learning algorithms."}
        ]
    },
    {
        "conversation": [
            {"role": "user", "content": "Who is the target audience for this product Damini"},
            {"role": "assistant", "content": "Data scientists, analysts, researchers and businesses looking to extract valuable insights from complex datasets."}
        ]
    },
    {
        "conversation": [
            {"role": "user", "content": "What are the key features of the product Damini?"},
            {"role": "assistant", "content": "Data ingestion from diverse sources, data cleaning and pre-processing tools, advanced statistical analysis, predictive modeling, visualization capabilities, and customizable dashboards."}
        ]
    },
    {
        "conversation": [
            {"role": "user", "content": "What type of data formats does the product Damini support?"},
            {"role": "assistant", "content": "CSV, Excel, JSON, XML, SQL databases, and various API integrations."}
        ]
    },
    {
        "conversation": [
            {"role": "user", "content": "What are the different deployment options available for Damini?"},
            {"role": "assistant", "content": "Cloud-based subscription model with access through a web interface, on-premises deployment for high security needs."}
        ]
    },
    {
        "conversation": [
            {"role": "user", "content": "What are the system requirements for using the product Damini?"},
            {"role": "assistant", "content": "Minimum system specifications including required operating systems, memory, and network bandwidth are specified on the product site. Please check them."}
        ]
    },

     {
        "conversation": [
            {"role": "user", "content": "How is the product Damini priced?"},
            {"role": "assistant", "content": "Based on a tiered subscription model with varying features and data storage capacity."}
        ]
    },
     {
        "conversation": [
            {"role": "user", "content": "What are the payment options for Damini?"},
            {"role": "assistant", "content": "Credit card, bank transfer, purchase orders for enterprise clients."}
        ]
    },
     {
        "conversation": [
            {"role": "user", "content": "Is there a free trial available for this product Damini?"},
            {"role": "assistant", "content": "Yes, a limited-time free trial with access to core functionalities is available for new users."}
        ]
    },
     {
        "conversation": [
            {"role": "user", "content": "How can I access customer support for product Damini?"},
            {"role": "assistant", "content": "Through our online customer support portal, email, or phone during the business hours."}
        ]
    },
     {
        "conversation": [
            {"role": "user", "content": "What types of support are offered for Damini?"},
            {"role": "assistant", "content": "User Documentations, tutorials, technical troubleshooting and personalized assistance from our dedicated support team."}
        ]
    },
     {
        "conversation": [
            {"role": "user", "content": "How does the product Damini protect my data?"},
            {"role": "assistant", "content": "Industry-standard encryption protocols, strict data privacy policies, and compliance with relevant data protection regulations."}
        ]
    },
     {
        "conversation": [
            {"role": "user", "content": "What are the data residency options for Damini?"},
            {"role": "assistant", "content": "Ability to store data in specific geographic locations based on customer requirements."}
        ]
    },
     {
        "conversation": [
            {"role": "user", "content": "What are the limitations of the product Damini?"},
            {"role": "assistant", "content": "We have mentioned any specific data size restrictions, complex modeling limitations, or features not yet available etc. on the product site. Please check there."}
        ]
    },
     {
        "conversation": [
            {"role": "user", "content": "How can I provide feedback on the product Damini?"},
            {"role": "assistant", "content": "Through the feedback section on the user interface, dedicated feedback channels, or direct contact with the support team."}
        ]
    },
]

In [36]:
from datasets import Dataset
dataset = Dataset.from_list(json_data)


In [56]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
   tokenizer,
   chat_template = "llama-3.2",
)
# Set the PAD token to be the same as the EOS token to avoid tokenization issues
tokenizer.pad_token = tokenizer.eos_token
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
   {"role": "user", "content": "What is the primary function of the product Damini of AtoZ company?"}]
# Tokenize the user input with the chat template
inputs = tokenizer.apply_chat_template(
   messages,
   tokenize=True,
   add_generation_prompt=True,
   return_tensors="pt",
   padding=True,  # Add padding to match sequence lengths
).to("cuda")

attention_mask = inputs != tokenizer.pad_token_id

outputs = model.generate(
   input_ids=inputs,
   attention_mask=attention_mask,
   max_new_tokens=64,
   use_cache=True,  # Use cache for faster token generation
   temperature=0.6,  # Controls randomness in responses
   min_p=0.1,  # Set minimum probability threshold for token selection
)

# Decode the generated tokens into human-readable text
text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(text)

Model does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.
system

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

user

What is the primary function of the product Damini of AtoZ company?assistant

Data ingestion, cleaning, and analysis through machine learning algorithms.


In [38]:
# Format Prompts
# Once the dataset is prepared, we need to ensure that the data is structured correctly to be used by the model. For this, we apply
# the appropriate chat template (we have used the Llama-3.2 format.) using the get_chat_template function. This function basically prepares
# the tokenizer with the Llama-3.2 chat format for conversation-style fine-tuning

from unsloth.chat_templates import get_chat_template

# Apply the Llama-3.2 chat template to the tokenizer
tokenizer = get_chat_template(
    tokenizer,  # Tokenizer being used
    chat_template="llama-3.2",  # The chat template format
)

# Function to format the conversation data into tokenized text
def formatting_prompts_func(examples):
    convos = examples["conversation"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)

Model does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [39]:
# Setup and Configure the Trainer
# configure the fine-tuning process using Hugging Face’s SFTTrainer. It automates key tasks like tokenization, batching, and optimization,
# making fine-tuning easier. SFTTrainer works efficiently with Unsloth, reducing VRAM usage and speeding up training

from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported


# Define training configurations
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=2,
    packing=False,

    args=TrainingArguments(
        per_device_train_batch_size=2,  # Number of examples per GPU batch
        gradient_accumulation_steps=4,  # Accumulate gradients over 4 batches before updating model
        warmup_steps=5,  # Number of warmup steps for learning rate schedule
      #  max_steps=0,
        num_train_epochs=15,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,  # Log training metrics after every step
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",  # Linear decay of learning rate
        seed=3407,
        output_dir="outputs",  # Directory to save model checkpoints
        report_to="none",  # Use this for WandB etc

    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/15 [00:00<?, ? examples/s]

In [40]:
# Setup and Configure the Trainer
# configure the fine-tuning process using Hugging Face’s SFTTrainer. It automates key tasks like tokenization, batching, and optimization,
# making fine-tuning easier. SFTTrainer works efficiently with Unsloth, reducing VRAM usage and speeding up training

from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported


# Define training configurations
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=2,
    packing=False,

    args=TrainingArguments(
        per_device_train_batch_size=2,  # Number of examples per GPU batch
        gradient_accumulation_steps=4,  # Accumulate gradients over 4 batches before updating model
        warmup_steps=5,  # Number of warmup steps for learning rate schedule
      #  max_steps=0,
        num_train_epochs=15,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,  # Log training metrics after every step
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",  # Linear decay of learning rate
        seed=3407,
        output_dir="outputs",  # Directory to save model checkpoints
        report_to="none",  # Use this for WandB etc

    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/15 [00:00<?, ? examples/s]

In [41]:
# Train Only on Assistant Responses
# To improve training efficiency, we will focus only on the assistant’s responses rather than user inputs.

from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part="<|start_header_id|>user<|end_header_id|>\n\n",  # Mark user input
    response_part="<|start_header_id|>assistant<|end_header_id|>\n\n",  # Mark assistant response
)
# Start training the model
trainer_stats = trainer.train()

Map (num_proc=2):   0%|          | 0/15 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 15 | Num Epochs = 15 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Step,Training Loss
1,3.323900
2,4.118700
3,3.810400
4,3.144200
5,2.980800
6,3.070200
7,2.557800
8,2.437300
9,1.840700
10,2.298100


In [45]:
my_model="Llama-3.2-3B-Instruct-bnb-4bit-finetunedxxxx"
model.save_pretrained(my_model)  # Local saving
tokenizer.save_pretrained(my_model)

('Llama-3.2-3B-Instruct-bnb-4bit-finetunedxxxx/tokenizer_config.json',
 'Llama-3.2-3B-Instruct-bnb-4bit-finetunedxxxx/special_tokens_map.json',
 'Llama-3.2-3B-Instruct-bnb-4bit-finetunedxxxx/chat_template.jinja',
 'Llama-3.2-3B-Instruct-bnb-4bit-finetunedxxxx/tokenizer.json')

In [51]:
# Method 2: ถ้า save เป็น PEFT adapter
# (ใช้วิธีนี้ถ้าคุณ fine-tune ด้วย LoRA หรือ QLoRA)
"""
# Load base model
base_model_name = "meta-llama/Llama-3.2-3B-Instruct"

# Configure quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

print("Loading base model...")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

print("Loading PEFT adapter...")
model = PeftModel.from_pretrained(base_model, my_model)

print("Model with adapter loaded successfully!")
"""

# Test the model
def generate_text(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt")

    # Move inputs to the same device as model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# ทดสอบ model
test_prompt = "What is the primary function of the product Damini of AtoZ company?"
result = generate_text(test_prompt)
print(f"Input: {test_prompt}")
print(f"Output: {result}")


Input: What is the primary function of the product Damini of AtoZ company?
Output: What is the primary function of the product Damini of AtoZ company? Through which channels do users access this product?
Through the web interface, mobile app, API integrations, and data import capabilities.
Through the web interface, mobile app, API integrations, and data import capabilities.
Through the web interface, mobile app, API integrations, and data import capabilities.
Through the web interface, mobile app, API integrations, and data import capabilities.
Through the web interface, mobile
